<a href="https://colab.research.google.com/github/mangeshkakad/tigergraph_globalwarming/blob/main/TigerGraph_RunnerV0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install TigerGraph python Module

In [2]:
!pip install -q pyTigerGraph

# Import required modules to interact with TigerGraph Cloud SaaS

In [3]:
# Imports
import pyTigerGraph as tg
import json
import pandas as pd

In [ ]:
pd.set_option('max_columns', None)
def pprint(input):
  print(json.dumps(input, indent=2))

# Connection to tgcloud.io to interact with Solution

In [4]:
# Connection parameters
hostName = "https://e58ef4c275ea43059948c09089d35d23.i.tgcloud.io/"
userName = "tigergraph"
password = ""

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print("Connected")

Connected


# Create Vertex and Edges from python module directly rather than manually doing it through TigerGraph console

In [ ]:
conn.gsql('''
CREATE VERTEX CO2(PRIMARY_ID id STRING, co_value FLOAT, Year INT,Month INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Ocean_Heat(PRIMARY_ID id STRING, Ocean_heat FLOAT, Year INT,Month INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Arctic_Sea_Ice_Extent(PRIMARY_ID id STRING, Extent FLOAT, Year INT,Month INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Temperature_Anomaly(PRIMARY_ID id STRING, Temp_Anomaly FLOAT, Year INT,Month INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Year(PRIMARY_ID year INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Month(PRIMARY_ID month INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
''')

'Successfully created vertex types: [CO2].\nSuccessfully created vertex types: [Ocean_Heat].\nSuccessfully created vertex types: [Arctic_Sea_Ice_Extent].\nSuccessfully created vertex types: [Temperature_Anomaly].\nSuccessfully created vertex types: [Year].\nSuccessfully created vertex types: [Month].'

In [ ]:
conn.gsql('''
CREATE DIRECTED EDGE co2_in_year(FROM CO2, TO Year) WITH REVERSE_EDGE="co2_reverse_in_year"
CREATE DIRECTED EDGE co2_during_months(FROM CO2, TO Month) WITH REVERSE_EDGE="co2_reverse_during_months"
CREATE DIRECTED EDGE heat_in_year(FROM Ocean_Heat, TO Year) WITH REVERSE_EDGE="heat_reverse_in_year"
CREATE DIRECTED EDGE heat_during_months(FROM Ocean_Heat, TO Month) WITH REVERSE_EDGE="heat_reverse_during_months"
CREATE DIRECTED EDGE extent_in_year(FROM Arctic_Sea_Ice_Extent, TO Year) WITH REVERSE_EDGE="extent_reverse_in_year"
CREATE DIRECTED EDGE extent_during_months(FROM Arctic_Sea_Ice_Extent, TO Month) WITH REVERSE_EDGE="extent_reverse_during_months"
CREATE DIRECTED EDGE temp_in_year(FROM Temperature_Anomaly, TO Year) WITH REVERSE_EDGE="temp_reverse_in_year"
CREATE DIRECTED EDGE temp_during_months(FROM Temperature_Anomaly, TO Month) WITH REVERSE_EDGE="temp_reverse_during_months"

''')

'Successfully created edge types: [co2_in_year].\nSuccessfully created reverse edge types: [co2_reverse_in_year].\nSuccessfully created edge types: [co2_during_months].\nSuccessfully created reverse edge types: [co2_reverse_during_months].\nSuccessfully created edge types: [heat_in_year].\nSuccessfully created reverse edge types: [heat_reverse_in_year].\nSuccessfully created edge types: [heat_during_months].\nSuccessfully created reverse edge types: [heat_reverse_during_months].\nSuccessfully created edge types: [extent_in_year].\nSuccessfully created reverse edge types: [extent_reverse_in_year].\nSuccessfully created edge types: [extent_during_months].\nSuccessfully created reverse edge types: [extent_reverse_during_months].\nSuccessfully created edge types: [temp_in_year].\nSuccessfully created reverse edge types: [temp_reverse_in_year].\nSuccessfully created edge types: [temp_during_months].\nSuccessfully created reverse edge types: [temp_reverse_during_months].'

# Create Global Climate Change graph

In [ ]:
conn.gsql('''
CREATE GRAPH Global_Climate_Change(CO2, Ocean_Heat, Arctic_Sea_Ice_Extent, Temperature_Anomaly, Year, Month, co2_in_year, co2_during_months, heat_in_year,heat_during_months,extent_in_year,extent_during_months,temp_in_year,temp_during_months)
''')

'The graph Global_Climate_Change is created.'

# Create secret token to connect through API calls

In [5]:
graphName = "Global_Climate_Change"
conn.graphname = graphName
secret = conn.createSecret()
token = conn.getToken(secret, setToken=True)
print(token[0])
conn.apiToken = token
print(token[0])

8g700nnvcv2r1ngkr2put5lb0pvkaa2f
8g700nnvcv2r1ngkr2put5lb0pvkaa2f


# To upload data file into Google Colab

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Arctic_Sea_Ice_Extent.csv to Arctic_Sea_Ice_Extent.csv
Saving CO2.csv to CO2.csv
Saving OCEAN_HEAT.CSV to OCEAN_HEAT.CSV
Saving Sea_Level.csv to Sea_Level.csv
Saving Temperature_Anomaly.csv to Temperature_Anomaly.csv


# Verify with header that files are uploaded correctly

In [ ]:
!head CO2.csv

Year,Month,CO2
1958,3,315.7
1958,4,317.45
1958,5,317.51
1958,6,317.24
1958,7,315.86
1958,8,314.93
1958,9,313.2
1958,10,312.43
1958,11,313.33


# Create JOB to load datapoints to Vertex and Edges

In [ ]:
conn.gsql('''
CREATE LOADING JOB data_load_co21 FOR GRAPH Global_Climate_Change {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX CO2 VALUES(gsql_concat($0,$1), $0,$1,$2) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }
''')

'Successfully created loading jobs: [data_load_co21].'

In [ ]:
conn.gsql('''
CREATE LOADING JOB data_load_oceanheat FOR GRAPH Global_Climate_Change {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Ocean_Heat VALUES(gsql_concat($0,$1), $0,$1,$2) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }
''')

'Successfully created loading jobs: [data_load_oceanheat].'

In [ ]:
conn.gsql('''
CREATE LOADING JOB data_load_seaiceextent FOR GRAPH Global_Climate_Change {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Arctic_Sea_Ice_Extent VALUES(gsql_concat($0,$1),$0,$1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }
''')

'Successfully created loading jobs: [data_load_seaiceextent].'

In [ ]:
conn.gsql('''
CREATE LOADING JOB data_load_tempanomaly FOR GRAPH Global_Climate_Change {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Temperature_Anomaly VALUES(gsql_concat($0,$1), $0,$1,$2) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }
''')

'Successfully created loading jobs: [data_load_tempanomaly].'

# Upload Carbon Footprint dataset from csv file

In [ ]:
import os

dataFile = 'CO2.csv'
results = conn.uploadFile(dataFile, fileTag='MyDataSource', jobName='data_load_co21')
print(json.dumps(results, indent=2))


In [ ]:
conn.gsql(
    '''
    USE GRAPH Global_Climate_Change
    CREATE QUERY addYearsAndMonths() FOR GRAPH Global_Climate_Change { 
      MinAccum<INT> @currentYear;
      MaxAccum<INT> @currentMonth;
      co2_value = {CO2.*};
      
      noMonth = SELECT r FROM co2_value:r
        WHERE
          r.OUTDEGREE("co2_during_months") == 0
        ACCUM
          r.@currentMonth = r.Month
        POST-ACCUM
          INSERT INTO co2_during_months (FROM, TO)VALUES (r.id, r.@currentMonth);
      
      noYear = SELECT r FROM co2_value:r
        WHERE
          r.OUTDEGREE("co2_in_year") == 0
        ACCUM
          r.@currentYear = r.Year
        POST-ACCUM
          INSERT INTO co2_in_year (FROM, TO)VALUES (r.id, r.@currentYear);
      
      PRINT noYear;
    }
    INSTALL QUERY addYearsAndMonths
    '''
)

'Using graph \'Global_Climate_Change\'\nSuccessfully created queries: [addYearsAndMonths].\nStart installing queries, about 1 minute ...\naddYearsAndMonths query: curl -X GET \'https://127.0.0.1:9000/query/Global_Climate_Change/addYearsAndMonths\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

# Upload Ocean heat dataset from csv file

In [ ]:
conn.gsql(
    '''
    USE GRAPH Global_Climate_Change
    CREATE QUERY addYearsAndMonths_1() FOR GRAPH Global_Climate_Change { 
      MinAccum<INT> @currentYear;
      MaxAccum<INT> @currentMonth;
      oh_value = {Ocean_Heat.*};
      
      noMonth = SELECT r FROM oh_value:r
        WHERE
          r.OUTDEGREE("heat_during_months") == 0
        ACCUM
          r.@currentMonth = r.Month
        POST-ACCUM
          INSERT INTO heat_during_months (FROM, TO)VALUES (r.id, r.@currentMonth);
      
      noYear = SELECT r FROM oh_value:r
        WHERE
          r.OUTDEGREE("heat_in_year") == 0
        ACCUM
          r.@currentYear = r.Year
        POST-ACCUM
          INSERT INTO heat_in_year (FROM, TO)VALUES (r.id, r.@currentYear);
      
      PRINT noYear;
    }
    INSTALL QUERY addYearsAndMonths_1
    '''
)

'Using graph \'Global_Climate_Change\'\nSuccessfully created queries: [addYearsAndMonths_1].\nStart installing queries, about 1 minute ...\naddYearsAndMonths_1 query: curl -X GET \'https://127.0.0.1:9000/query/Global_Climate_Change/addYearsAndMonths_1\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [ ]:
conn.runInstalledQuery('addYearsAndMonths', params={})

# Upload Tempeature Anomaly dataset from csv file

In [ ]:
conn.gsql(
    '''
    USE GRAPH Global_Climate_Change
    CREATE QUERY addYearsAndMonths_2() FOR GRAPH Global_Climate_Change { 
      MinAccum<INT> @currentYear;
      MaxAccum<INT> @currentMonth;
      temp_value = {Temperature_Anomaly.*};
      
      noMonth = SELECT r FROM temp_value:r
        WHERE
          r.OUTDEGREE("temp_during_months") == 0
        ACCUM
          r.@currentMonth = r.Month
        POST-ACCUM
          INSERT INTO temp_during_months (FROM, TO)VALUES (r.id, r.@currentMonth);
      
      noYear = SELECT r FROM temp_value:r
        WHERE
          r.OUTDEGREE("temp_in_year") == 0
        ACCUM
          r.@currentYear = r.Year
        POST-ACCUM
          INSERT INTO temp_in_year (FROM, TO)VALUES (r.id, r.@currentYear);
      
      PRINT noYear;
    }
    INSTALL QUERY addYearsAndMonths_2
    '''
)

'Using graph \'Global_Climate_Change\'\nSuccessfully created queries: [addYearsAndMonths_2].\nStart installing queries, about 1 minute ...\naddYearsAndMonths_2 query: curl -X GET \'https://127.0.0.1:9000/query/Global_Climate_Change/addYearsAndMonths_2\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [ ]:
conn.runInstalledQuery('addYearsAndMonths_1', params={})

In [ ]:
conn.runInstalledQuery('addYearsAndMonths_2', params={})

# Upload Arctic Sea Ice Extent dataset from csv file

In [ ]:
conn.gsql(
    '''
    USE GRAPH Global_Climate_Change
    CREATE QUERY addYearsAndMonths_3() FOR GRAPH Global_Climate_Change { 
      MinAccum<INT> @currentYear;
      MaxAccum<INT> @currentMonth;
      extent_value = {Arctic_Sea_Ice_Extent.*};
      
      noMonth = SELECT r FROM ea_Ice_Extent.*};
      
      noMonth = SELECT r FROM extent_value:r
        WHERE
          r.OUTDEGREE("extent_during_months") == 0
        ACCUM
          r.@currentMonth = r.Month
        POST-ACCUM
          INSERT INTO extent_during_months (FROM, TO)VALUES (r.id, r.@currentMonth);
      
      noYear = SELECT r FROM extent_value:r
        WHERE
          r.OUTDEGREE("extent_in_year") == 0
        ACCUM
          r.@currentYear = r.Year
        POST-ACCUM
          INSERT INTO extent_in_year (FROM, TO)VALUES (r.id, r.@currentYear);
      
      PRINT noYear;
    }
    INSTALL QUERY addYearsAndMonths_3
    '''
)

'Using graph \'Global_Climate_Change\'\nSuccessfully created queries: [addYearsAndMonths_3].\nStart installing queries, about 1 minute ...\naddYearsAndMonths_3 query: curl -X GET \'https://127.0.0.1:9000/query/Global_Climate_Change/addYearsAndMonths_3\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [ ]:
conn.runInstalledQuery('addYearsAndMonths_3', params={})

# Create Query to extract data from Global_Climate_Change graph for Machine Learning

In [ ]:
conn.gsql(
    '''
    USE GRAPH Global_Climate_Change
    CREATE QUERY getDataForMLModelv6(SET <INT> mYear) FOR GRAPH Global_Climate_Change SYNTAX v2 {

          temp = SELECT ta FROM Temperature_Anomaly:ta WHERE ta.Year IN mYear ORDER BY ta.Month;
          PRINT temp;
      
          co2 = SELECT co FROM CO2:co WHERE co.Year IN mYear ORDER BY co.Month;
          PRINT co2;
      
          extent = SELECT ae FROM Arctic_Sea_Ice_Extent:ae WHERE ae.Year IN mYear ORDER BY ae.Month;
          PRINT extent;
      
          heat = SELECT oh FROM Ocean_Heat:oh WHERE oh.Year IN mYear ORDER BY oh.Month; 
          PRINT heat;

        }
    '''
)

"Using graph 'Global_Climate_Change'\nSuccessfully created queries: [getDataForMLModelv6]."

In [ ]:
conn.gsql('''
INSTALL QUERY getDataForMLModelv6
''')

'Start installing queries, about 1 minute ...\ngetDataForMLModelv6 query: curl -X GET \'https://127.0.0.1:9000/query/Global_Climate_Change/getDataForMLModelv6?mYear=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [ ]:
res = conn.runInstalledQuery('getDataForMLModelv6', params={"mYear":2008,"mYear":2009})

In [ ]:
temp = res[0]["temp"]
co2 = res[1]["co2"]
extent = res[2]["extent"]
heat = res[3]["heat"]

In [ ]:
conn.vertexSetToDataFrame(temp)

,v_id,id,Temp_Anomaly,Year,Month
0,20091,20091,0.64,2009,1


In [ ]:
conn.vertexSetToDataFrame(co2)


,v_id,id,co_value,Year,Month
0,20131,20131,395.78000,2013,1
1,20132,20132,397.03000,2013,2
2,20133,20133,397.66000,2013,3
3,20134,20134,398.64001,2013,4
4,20135,20135,400.01999,2013,5
5,20136,20136,398.81000,2013,6
6,20137,20137,397.51001,2013,7
7,20138,20138,395.39001,2013,8
8,20139,20139,393.72000,2013,9
9,201310,201310,393.89999,2013,10


In [ ]:
conn.vertexSetToDataFrame(extent)


,v_id,id,Extent,Year,Month
0,20131,20131,5.21,2013,1


In [ ]:
conn.vertexSetToDataFrame(heat)

,v_id,id,Ocean_heat,Year,Month
0,20131,20131,14.90705,2013,1
1,20132,20132,15.04299,2013,2
2,20133,20133,15.07405,2013,3
3,20134,20134,15.18344,2013,4
4,20135,20135,15.18823,2013,5
5,20136,20136,15.05973,2013,6
6,20137,20137,15.23519,2013,7
7,20138,20138,15.39749,2013,8
8,20139,20139,15.41166,2013,9
9,201310,201310,15.38795,2013,10


# Create Query to Insert Data which can be used from python code to insert forecasted data sets

In [ ]:
conn.gsql(
    '''
    USE GRAPH Global_Climate_Change
    CREATE QUERY Insert_Records(INT year , INT month, FLOAT co_value,FLOAT extent, FLOAT temp_anomaly,FLOAT Ocean_heat) FOR GRAPH Global_Climate_Change { 
      INSERT INTO CO2 (PRIMARY_ID, co_value,Year,Month) VALUES (to_string(year)+to_string(month),co_value,year,month);
      INSERT INTO Arctic_Sea_Ice_Extent (PRIMARY_ID, Extent,Year,Month) VALUES (to_string(year)+to_string(month),extent,year,month);
      INSERT INTO Temperature_Anomaly (PRIMARY_ID, Temp_Anomaly,Year,Month) VALUES (to_string(year)+to_string(month),temp_anomaly,year,month);
      INSERT INTO Ocean_Heat (PRIMARY_ID, Ocean_heat,Year,Month) VALUES (to_string(year)+to_string(month),Ocean_heat,year,month);

      INSERT INTO Year (PRIMARY_ID) VALUES (year);
      INSERT INTO Month (PRIMARY_ID) VALUES (month);
    }
    INSTALL QUERY Insert_Records
    '''
)

'Using graph \'Global_Climate_Change\'\nSuccessfully created queries: [Insert_Records].\nStart installing queries, about 1 minute ...\nInsert_Records query: curl -X GET \'https://127.0.0.1:9000/query/Global_Climate_Change/Insert_Records?year=VALUE&month=VALUE&co_value=VALUE&extent=VALUE&temp_anomaly=VALUE&Ocean_heat=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

# Create Query to Delete Data

In [ ]:
conn.gsql(
    '''
    USE GRAPH Global_Climate_Change
    CREATE QUERY Delete_Records(SET <INT> month,INT year) FOR GRAPH Global_Climate_Change {
      C = {CO2.*};
      A = {Arctic_Sea_Ice_Extent.*};
      O = {Ocean_Heat.*};
      T = {Temperature_Anomaly.*};

      DELETE co FROM C:co WHERE co.Year == year AND co.Month IN month;
      DELETE ie FROM A:ie WHERE ie.Year == year AND ie.Month IN month;
      DELETE oh FROM O:oh WHERE oh.Year == year AND oh.Month IN month;
      DELETE ta FROM T:ta WHERE ta.Year == year AND ta.Month IN month;

    }
    INSTALL QUERY Delete_Records
    '''
)

'Using graph \'Global_Climate_Change\'\nSuccessfully created queries: [Delete_Records].\nStart installing queries, about 1 minute ...\nDelete_Records query: curl -X GET \'https://127.0.0.1:9000/query/Global_Climate_Change/Delete_Records?month=VALUE&year=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

# Create Query to Select Forecasted data for given year and month

In [6]:
conn.gsql(
    '''
    USE GRAPH Global_Climate_Change
    CREATE QUERY Select_Records(INT year , INT month) FOR GRAPH Global_Climate_Change { 
      C = {CO2.*};
      A = {Arctic_Sea_Ice_Extent.*};
      O = {Ocean_Heat.*};
      T = {Temperature_Anomaly.*};
      Y = {Year.*};
      M = {Month.*};
  
      co_results = SELECT co FROM C:co -(co2_in_year>)- Y:y WHERE co.Year == year and co.Month == month;
      ie_results = SELECT ie FROM A:ie -(extent_in_year>)- Y:y WHERE ie.Year == year and ie.Month == month;
      oh_results = SELECT oh FROM O:oh -(heat_in_year>)- Y:y WHERE oh.Year == year and oh.Month == month;
      ta_results = SELECT ta FROM T:ta -(temp_in_year>)- Y:y WHERE ta.Year == year and ta.Month == month;
  
      PRINT co_results;
      PRINT ie_results;
      PRINT oh_results;
      PRINT ta_results;
    }
    INSTALL QUERY Select_Records
    '''
)

'Using graph \'Global_Climate_Change\'\nSuccessfully created queries: [Select_Records].\nStart installing queries, about 1 minute ...\nSelect_Records query: curl -X GET \'https://127.0.0.1:9000/query/Global_Climate_Change/Select_Records?year=VALUE&month=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'